In [24]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/C89346C23293_map.csv')

data = df.values[:,1:]
print data[0]
print data.shape
data = data.flatten().reshape(-1,4)
data = map(int,np.sum(data,axis=1).clip(max=1))
data = np.array(data).reshape(-1,24)
print data.shape
print data[:30]


[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0
 0.0 0.0 0.0 0.0 0.0 0.0]
(168, 96)
(168, 24)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 

In [25]:
# use 4 days to predict 1 day
PRE_DAYS = 4
ONE_DAY_DATA_LEN = 24
def make_data(X, pre_days=PRE_DAYS):
    xs = []
    ys = []
    data_len = len(X)/ONE_DAY_DATA_LEN
    print data_len
    for i in range(data_len-PRE_DAYS):
        start_idx = i*ONE_DAY_DATA_LEN
        x = X[start_idx:start_idx+pre_days*ONE_DAY_DATA_LEN]
        y = X[start_idx+pre_days*ONE_DAY_DATA_LEN:start_idx+(1+pre_days)*ONE_DAY_DATA_LEN]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

x,y = make_data(data.flatten())
print x.shape,y.shape

def split_train_test(x, y, n):
    x_train, x_test = x[0:-n], x[-n:]
    y_train, y_test = y[0:-n], y[-n:]
    return x_train, y_train, x_test, y_test

X_train, Y_train, X_test, Y_test = split_train_test(x, y, 30)
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.regularizers import l2, activity_l2

INPUT_LEN = ONE_DAY_DATA_LEN * PRE_DAYS

model = Sequential()
model.add(Activation('tanh', input_shape=(INPUT_LEN,)))
model.add(Dense(INPUT_LEN))
model.add(Activation('tanh', input_shape=(INPUT_LEN,)))
model.add(Dense(INPUT_LEN/2, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
model.add(Dropout(0.1))
model.add(Activation('tanh', input_shape=(INPUT_LEN/2,)))
model.add(Dense(INPUT_LEN/4, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
model.add(Dropout(0.1))
model.add(Activation('sigmoid', input_shape=(INPUT_LEN/4,)))
model.add(Dense(ONE_DAY_DATA_LEN))

model.compile(loss='mse', optimizer='rmsprop')

print 'compile done'

model.fit(X_train, Y_train, nb_epoch=100, verbose=1, batch_size=50, validation_data=(X_test, Y_test))

print 'fit done'

168
(164, 96) (164, 24)
(134, 96) (134, 24) (30, 96) (30, 24)
compile done
Train on 134 samples, validate on 30 samples
Epoch 1/100
134/134 [==============================] - 0s - loss: 1.4187 - val_loss: 0.2981
Epoch 2/100
134/134 [==============================] - 0s - loss: 1.2457 - val_loss: 0.2458
Epoch 3/100
134/134 [==============================] - 0s - loss: 1.1491 - val_loss: 0.2150
Epoch 4/100
134/134 [==============================] - 0s - loss: 1.0756 - val_loss: 0.1915
Epoch 5/100
134/134 [==============================] - 0s - loss: 1.0128 - val_loss: 0.1726
Epoch 6/100
134/134 [==============================] - 0s - loss: 0.9567 - val_loss: 0.1571
Epoch 7/100
134/134 [==============================] - 0s - loss: 0.9051 - val_loss: 0.1433
Epoch 8/100
134/134 [==============================] - 0s - loss: 0.8584 - val_loss: 0.1312
Epoch 9/100
134/134 [==============================] - 0s - loss: 0.8143 - val_loss: 0.1205
Epoch 10/100
134/134 [==============================

In [26]:

pred = model.predict(X_test).flatten().reshape(-1,1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
pred = scaler.fit_transform(pred).round().flatten()

truth = Y_test.flatten()

from sklearn import metrics
test = [0]*len(truth)
test[0] = 1

print metrics.accuracy_score(truth,pred)
print metrics.f1_score(truth,pred)
print metrics.accuracy_score(truth,test)
print metrics.f1_score(truth,test)


0.905555555556
0.546666666667
0.918055555556
0.0327868852459


In [27]:
# use 4 days to predict 1 data
WINDOW = 4*ONE_DAY_DATA_LEN
def make_data(X, offset = WINDOW):
    xs = []
    ys = []
    data_len = len(X)
    print data_len
    for i in range(data_len-offset):
        start_idx = i
        x = X[start_idx:start_idx+offset]
        y = X[start_idx+offset:start_idx+1+offset]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

x,y = make_data(data.flatten())
print x.shape,y.shape

def split_train_test(x, y, n):
    x_train, x_test = x[0:-n], x[-n:]
    y_train, y_test = y[0:-n], y[-n:]
    return x_train, y_train, x_test, y_test

X_train, Y_train, X_test, Y_test = split_train_test(x, y, 300)
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

4032
(3936, 96) (3936, 1)
(3636, 96) (3636, 1) (300, 96) (300, 1)


In [28]:


from xgboost import XGBClassifier

xgb = XGBClassifier(max_depth=3,n_estimators=100,silent=False,reg_alpha=1)
xgb.fit(X_train,Y_train)

plt.figure(figsize=(20,10))
pred = xgb.predict(X_test)
truth = Y_test

print 'test'
print metrics.accuracy_score(truth,pred)
print metrics.f1_score(truth,pred)

pred = xgb.predict(X_train)
truth = Y_train

print 'train'
print metrics.accuracy_score(truth,pred)
print metrics.f1_score(truth,pred)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test
0.956666666667
0.628571428571
train
0.934543454345
0.536964980545
